In [ ]:
import os

import psycopg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from dotenv import load_dotenv, find_dotenv

In [ ]:
# подгружаем .env
load_dotenv()

In [ ]:
TABLE_NAME = "users_churn" # таблица с данными в postgres 

EXPERIMENT_NAME = "churn_experiment_imartnv"
RUN_NAME = "eda"

# Директория для хранения артефактов
ASSETS_DIR = "assets"

os.makedirs(ASSETS_DIR, exist_ok=True)

# Настрофка отображения
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

sns.set_style("white")
sns.set_theme(style="whitegrid") 

In [ ]:
# Подключение к базе и получение данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

In [ ]:
df.head(2)

In [ ]:
# Настройки для графиков
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
fig.set_size_inches(12, 8, forward=True) 
fig.tight_layout(pad=2)

# Первый график
x = "type"
y = "customer_id"
stat = ["count"]

agg_df = df.groupby(x)[y].agg(stat).reset_index()

sns.barplot(data=agg_df, x=x, y=stat[0], ax=axs[0, 0]) 
axs[0, 0].set_title(f'Count {y} by {x} in train dataframe')

# Второй график
x = "payment_method"
y = "customer_id"
stat = ["count"]

agg_df = df.groupby(x)[y].agg(stat).reset_index()

sns.barplot(data=agg_df, x=x, y=stat[0], ax=axs[1, 0]) 
axs[1, 0].set_title(f'Count {y} by {x} in train dataframe')
axs[1, 0].set_xticklabels(agg_df[x], rotation=40)  # Используем данные из agg_df для меток


# Третий график
x = "internet_service"
y = "customer_id"
stat = ["count"]

agg_df = df.groupby(x)[y].agg(stat).reset_index()

sns.barplot(data=agg_df, x=x, y=stat[0], ax=axs[0, 1]) 
axs[0, 1].set_title(f'Count {y} by {x} in train dataframe')
axs[0, 1].set_xticklabels(agg_df[x], rotation=30)  # Используем данные из agg_df для меток


# Чертвертый график
x = "gender"
y = "customer_id"
stat = ["count"]

agg_df = df.groupby(x)[y].agg(stat).reset_index()

sns.barplot(data=agg_df, x=x, y=stat[0], ax=axs[1, 1]) 
axs[1, 1].set_title(f'Count {y} by {x} in train dataframe')
axs[1, 1].set_xticklabels(agg_df[x], rotation=30)  # Используем данные из agg_df для меток


plt.savefig(os.path.join(ASSETS_DIR, 'cat_features_1'))

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
encoder = LabelEncoder()

In [ ]:
x = "customer_id"
binary_columns = [
    "online_security", 
    "online_backup", 
    "device_protection", 
    "tech_support",
    "streaming_tv",
    "streaming_movies",
    "senior_citizen",
    "partner",
    "dependents",
    "multiple_lines",
    "paperless_billing"
]
stat = ["count"]

print(df.groupby(binary_columns).agg(stat[0])[x].reset_index().sort_values(by=x, ascending=False).head(5))

In [ ]:
for column in binary_columns:
    df[column] = encoder.fit_transform(df[column].fillna("No"))

In [ ]:
df.head()

In [ ]:
heart_map = df[binary_columns].apply(pd.Series.value_counts).T

In [ ]:
sns.heatmap(heart_map)
plt.savefig(os.path.join(ASSETS_DIR, 'cat_features_2'))

In [ ]:
# инициализация переменной для названия колонки
x = "begin_date"

# список колонок, для которых будут вычисляться статистики
charges_columns = [
    "monthly_charges",
    "total_charges",
]

# удаление пустых колонок для посчёта медианного значения
df.dropna(subset=charges_columns, how='any', inplace=True)

stats = ["mean", "median", lambda x: x.mode().iloc[0]]

charges_monthly_agg = df.groupby(x)['monthly_charges'].agg(stats).reset_index()
charges_monthly_agg.columns = [x, "monthly_mean", "monthly_median", "monthly_mode"]

charges_total_agg = df.groupby(x)['total_charges'].agg(stats).reset_index()
charges_total_agg.columns = [x, "total_mean", "total_median", "total_mode"]

In [ ]:
# создание объекта для отображения графиков (2 графика вертикально)
fig, axs = plt.subplots(2, 1)
# настройка отступов между графиками
fig.tight_layout(pad=2.5)
# установка размера фигуры
fig.set_size_inches(6.5, 5.5, forward=True)

# Построение линейных графиков для ежемесячных платежей
for col in ["monthly_mean", "monthly_median", "monthly_mode"]:
    sns.lineplot(data=charges_monthly_agg, x=x, y=col, ax=axs[0])

# Установка заголовка для верхнего графика
axs[0].set_title(f"Count statistics for {charges_columns[0]} by {x}")

# Построение линейных графиков для общих платежей
for col in ["total_mean", "total_median", "total_mode"]:
    sns.lineplot(data=charges_total_agg, x=x, y=col, ax=axs[1])

# Установка заголовка для нижнего графика
axs[1].set_title(f"Count statistics for {charges_columns[1]} by {x}")

plt.savefig(os.path.join(ASSETS_DIR, 'cat_features_3'))

In [ ]:
x = 'target'
stats = ['nunique']

In [ ]:
agg_df = df.groupby(x).agg(stats).reset_index().melt(id_vars=["target"], var_name="Feature", value_name="Unique Values")

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(
    data=agg_df,
    x="Feature",
    y="Unique Values",
    hue="target"
)
plt.xticks(rotation=45, ha='right')
plt.title("Количество уникальных значений по target")
plt.ylabel("Уникальные значения")
plt.xlabel("Признаки")
plt.legend(title="Target", loc="upper right")
plt.tight_layout()
plt.savefig(os.path.join(ASSETS_DIR, 'cat_features_4'))

In [ ]:
# подсчёт количества каждого уникального значения в колонке и сброс индекса для последующей визуализации
target_agg = df[x].value_counts().reset_index()

# создание столбчатой диаграммы для визуализации распределения целевой переменной
plt.figure(figsize=(12, 6))
sns.barplot(data=target_agg, x='target', y='count')
plt.xticks(rotation=45, ha='right')
plt.title(f"{x} total distribution")
plt.legend(title="Target", loc="upper right")
plt.tight_layout()
plt.savefig(os.path.join(ASSETS_DIR, 'cat_features_5'))

In [ ]:
x = "begin_date"
target = "target"
# определение статистики для агрегации
stat = ["count"]


In [ ]:
target_agg_by_date = df[df[target]==1].groupby(x)[target].agg(stat).reset_index()
# удаление мультиуровневости заголовков после агрегации и переименование для удобства
target_agg_by_date.columns = target_agg_by_date.columns
target_agg_by_date.columns = [x, "target_count"]

In [ ]:
target_agg = df.groupby([x,target])['customer_id'].agg(stat).reset_index()
target_agg.columns = [x,target ,"target_customer_id"]

In [ ]:
# расчёт суммы и количества для конверсии по датам
conversion_agg = df.groupby(x)[target].agg(['count','sum']).reset_index()
# вычисление коэффициента конверсии и округление до двух знаков
conversion_agg["conv"] = conversion_agg['sum']/conversion_agg['count']

In [ ]:
# аналогичный расчет конверсии, но с дополнительным разделением по полу
conversion_agg_gender = df.groupby([x,'gender'])[target].agg(['count','sum']).reset_index()
conversion_agg_gender["conv"] = conversion_agg_gender['sum']/conversion_agg_gender['count']

In [ ]:
fig, axs = plt.subplots(2,2)
fig.tight_layout(pad=1.6)
fig.set_size_inches(16.5, 12.5, forward=True) 

# визуализация общего количества целей по датам начала
sns.lineplot(data=target_agg_by_date, x=x, y='target_count', ax=axs[0, 0])
axs[0, 0].set_title("Target count by begin date")


# визуализация количества клиентов для каждого типа цели по датам
sns.lineplot(data=target_agg, x=x, y="target_customer_id", hue=target, ax=axs[0, 1])
axs[0, 1].set_title("Target count type by begin date")

# визуализация коэффициента конверсии по датам
sns.lineplot(data=conversion_agg, x=x, y="conv", ax=axs[1, 0])
axs[1, 0].set_title("Conversion value")


# визуализация коэффициента конверсии по датам с разделением по полу
sns.lineplot(data=conversion_agg_gender, x=x, y="conv",hue='gender' ,ax=axs[1, 1])
axs[1, 1].set_title("Conversion value by gender")

plt.savefig(os.path.join(ASSETS_DIR, 'target_by_date'))

In [ ]:
charges = ["monthly_charges", "total_charges"]
target = "target"

In [ ]:
sns.histplot(data=df, x=charges[0],hue='target', kde=True)

In [ ]:
fig, axs = plt.subplots(2, 1)
fig.tight_layout(pad=1.5)  # настройка отступов между подграфиками
fig.set_size_inches(6.5, 6.5, forward=True)  # установка размера фигуры

sns.histplot(data=df, x=charges[0],hue='target', kde=True, ax=axs[0])
axs[0].set_title(f"{charges[0]} distribution")

sns.histplot(data=df, x=charges[1],hue='target', kde=True, ax=axs[1])
axs[1].set_title(f"{charges[1]} distribution")

plt.savefig(os.path.join(ASSETS_DIR, 'chargest_by_target_dist'))

In [ ]:
ASSETS_DIR = 'assets'

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_artifacts(ASSETS_DIR) 
    
print("Run ID:", run_id)